## Imports

In [40]:
# Algorithm Imports
from random import random, randint
import numpy as np

# Functional Mapping Imports
from operator import add
from functools import reduce

## Test Individual Function
Create population member

In [18]:
def individual(length, min, max):
    return[randint(min,max) for x in np.arange(length)]


In [20]:
print(individual(5, 0, 100),
      individual(5, 0, 100),
      individual(5, 0, 100))

[62, 89, 69, 26, 43] [33, 89, 40, 62, 81] [15, 17, 51, 82, 26]


## Test Population Function
Create number of individuals (i.e. population)
* count: Number of individuals
* length: number of values per individual
* min: min value in individual list of values
* max: max value in individual list of values


In [17]:
def population(count, length, min, max):    
    return [individual(length, min, max) for x in np.arange(count)]


In [21]:
print(population(3, 5, 0, 100))
print(population(3, 5, 0, 100))
print(population(3, 5, 0, 100))

[[77, 48, 73, 88, 25], [62, 80, 80, 94, 82], [81, 24, 78, 82, 34]]
[[90, 3, 68, 2, 56], [12, 4, 1, 73, 88], [72, 93, 15, 97, 67]]
[[26, 87, 36, 69, 28], [61, 53, 8, 93, 31], [87, 43, 60, 64, 74]]


## Test Fitness Function
Determine Individual Fitness  <b><font color='red'>(Lower is Better)</font></b>
* individual: Individual Evaluated
* target: Sum of numbers that individuals aim for

In [23]:
def fitness(individual, target):
    sum = reduce(add, individual, 0)
    return abs(target - sum)

In [27]:
x1 = individual(5, 0, 100)
x2 = individual(5, 0, 100)
x3 = individual(5, 0, 100)

print('fitness = ', fitness(x1, 200))
print('fitness = ', fitness(x2, 200))
print('fitness = ', fitness(x3, 200))

fitness =  142
fitness =  54
fitness =  34


## Test Fitness Grading
Average fitness of population

In [29]:
def grade(pop, target):
    summed = reduce(add, (fitness(x, target) for x in pop), 0)
    return summed / (len(pop) * 1.0)

In [30]:
x4 = population(3, 5, 0, 100)
x5 = population(3, 5, 0, 100)
x6 = population(3, 5, 0, 100)

target = 200

print('Grade = ', grade(x4, target))
print('Grade = ', grade(x5, target))
print('Grade = ', grade(x6, target))

Grade =  22.333333333333332
Grade =  77.66666666666667
Grade =  37.666666666666664


## Evolution (Promoting genetic diversity)
1. For each generation, take a portion of best performing individuals judged by fitness function. Will be parents of next generation
* Randomly select lesseer individuals to be parents to promote diversity
* Can get stuck in local min/max
* Decrease likelihood of getting stuck with diversity
2. 'Breed Parents' to repopulate to desired sie
* Take top 20 individuals in population of 100
* Create 80 new children via breeding
* N/2 digits from father, N/2 digits from mother

<b><font color='red'>Important:</font></b>
One parent may breed multiple times, but should never be both mother and father

### Breed Example

In [39]:
father = [1, 2, 3, 4, 5, 6]
mother = [10, 20, 30, 40, 50 , 60]
child = father[:3] + mother[3:]
print('father:\t', father)
print('mother:\t', mother)
print('child:\t', child)

father:	 [1, 2, 3, 4, 5, 6]
mother:	 [10, 20, 30, 40, 50, 60]
child:	 [1, 2, 3, 40, 50, 60]


### Breed Function

In [63]:
def breed(pop, parents):
    # Crossover parents to create children
    parents_length = len(parents)
    desired_length = len(pop) - parents_length
    children = []
    
    while len(children) < desired_length:
        male = randint(0, parents_length - 1)
        female = randint(0, parents_length - 1)
        if male != female:
            male = parents[male]
            female = parents[female]
            
            # Choose either M/F Floor
            half = int(len(male) / 2)
            child = male[:half] + female[half:]
            children.append(child)
    
    return children

### Mutate Function

In [56]:
# Mutate
def mutate(population, chance_to_mutate):
#     chance_to_mutate = 0.01
    for i in population:
        if chance_to_mutate > random():
            place_to_modify = randint(0, len(i) - 1)
            # Not Ideal: Restricts range of values
            # Function is unaware of individual min/max values
            i[place_to_modify] = randint(min(i), max(i))

### Evolve (1 Generation)

In [57]:
# Evolve (Breed, Merge, Mutate)
def evolve(pop, target, retain, random_select, chance_to_mutate):
    # Run Fitness on Population, Sort by Fitness
    # Select Parents from population with highest fitness
    graded = [(fitness(x, target), x) for x in pop]
    graded = [x[1] for x in sorted(graded)]
    retain_length = int(len(graded) * retain)
    parents = graded[:retain_length]
    
    # Randomly add other individuals to promote genetic diversity
    for individual in graded[retain_length:]:
        if random_select > random():
            parents.append(individual)
    
    # Mutate some individuals
    mutate(pop, chance_to_mutate)
    
    # Crossover parents to create children
    children = breed(pop, parents)
    parents.extend(children)
    
    return parents 

## Testing it Out

In [64]:
target = 371 # Fitness Target
p_count = 100 # Population Count
retain = 0.2 # Greatest Fitness percent to retain
random_select = 0.05 # Percent Chance of random addition (diverstiy)
chance_to_mutate = 0.01 # Percent Chance of inidividual mutation

i_length = 5
i_min = 0
i_max = 100

p = population(p_count, i_length, i_min, i_max)
fitness_history = [grade(p, target),]

for i in np.arange(100):
    p = evolve(p, target, retain, random_select, chance_to_mutate)
    fitness_history.append(grade(p, target))
    
for datum in fitness_history:
    print(datum)

123.75
52.78
33.06
25.84
20.35
17.75
4.69
3.47
0.56
0.45
0.65
0.7
0.1
0.15
0.5
0.6
0.55
0.25
0.85
1.0
1.45
1.95
1.65
1.45
0.42
0.12
0.0
0.0
0.42
0.42
0.0
0.34
0.21
0.0
0.0
0.0
0.0
0.0
0.86
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.76
0.0
0.0
0.0
0.0
0.0
0.0
0.44
0.0
0.0
0.0
0.0
0.72
0.28
0.2
0.12
0.0
0.36
0.08
0.0
0.06
0.0
0.0
0.0
0.0
0.0
2.22
0.0
0.23
0.0
2.05
0.0
0.24
0.0
0.0
0.0
0.0
4.94
1.32
0.0
0.0
0.0
0.0
0.54
1.08
0.9
1.0
0.8
0.0
0.0
0.0
0.0
0.0
0.0
